<a href="https://colab.research.google.com/github/unie12/recommendation/blob/main/contents_cf_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

movielens 20m 데이터셋 사용

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
movies_df = pd.read_csv('/content/movie.csv')
genome_tags_df = pd.read_csv('/content/genome_tags.csv')
genome_scores_df = pd.read_csv('/content/genome_scores.csv')

In [ ]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
genome_tags_df.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [ ]:
genome_scores_df.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [ ]:
merged_genome = genome_scores_df.merge(genome_tags_df, on='tagId', how='left')

In [ ]:
merged_genome.head()

,movieId,tagId,relevance,tag
0,1,1,0.02500,007
1,1,2,0.02500,007 (series)
2,1,3,0.05775,18th century
3,1,4,0.09675,1920s
4,1,5,0.14675,1930s


In [ ]:
top_tags = merged_genome[merged_genome['relevance'] > 0.5]

In [ ]:
top_tags.reset_index(drop=True, inplace=True)

In [ ]:
top_tags.head()

,movieId,tagId,relevance,tag
0,1,11,0.57700,3d
1,1,19,0.67050,action
2,1,29,0.89200,adventure
3,1,30,0.67625,affectionate
4,1,61,0.54475,animal movie


In [ ]:
top_tags['tag'].where(top_tags['movieId'] == 1).dropna()

,tag
0,3d
1,action
2,adventure
3,affectionate
4,animal movie
...,...
77,very funny
78,very good
79,visual
80,whimsical


In [ ]:
list(top_tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)))[1]

'action adapted from:book adventure animal movie animals based on a book based on a video game based on book big budget chase childhood children comedy computer animation computer game destiny dynamic cgi action effects entertaining exciting family fantasy fantasy world fun fun movie good good action great great movie happy ending jungle kids kids and family lions magic mentor monkey nostalgic original pg-13 predictable runaway saturn award (best special effects) scary special effects spiders story supernatural video game video games videogame'

In [ ]:
grouped_tags = top_tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

In [ ]:
grouped_tags.head()

,movieId,tag
0,1,3d action adventure affectionate animal movie ...
1,2,action adapted from:book adventure animal movi...
2,3,chase comedy crappy sequel destiny family fish...
3,4,adultery betrayal chick flick divorce feel goo...
4,5,catastrophe chase comedy crappy sequel culture...


In [ ]:
print(movies_df['movieId'].nunique())
movies_df.tail()

27280


,movieId,title,genres
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)
27277,131262,Innocence (2014),Adventure|Fantasy|Horror
27278,132000,Babylon (2021),Comedy|Drama
27279,132001,Decision to Leave (2022),Mystery|Thriller|Romance


In [ ]:
print(grouped_tags['movieId'].size)
print(movies_df['movieId'].size)
print("Uniques in movies_df : ", movies_df['movieId'].nunique())

10381
27280
Uniques in movies_df :  27280


In [ ]:
# print(movies_df.merge(grouped_tags, on='movieId', how='inner').shape)
# final_df = movies_df.merge(grouped_tags, on='movieId', how='inner')
print(movies_df.merge(grouped_tags, on='movieId', how='left').shape)
final_df = movies_df.merge(grouped_tags, on='movieId', how='left')

(27280, 4)


In [ ]:
final_df = final_df[['movieId', 'title', 'genres', 'tag']]

In [ ]:
final_df.sample(13)

,movieId,title,genres,tag
26493,127134,A Walk in the Woods (2015),Adventure|Comedy|Drama,adaptation adapted from:book adventure aging b...
24406,115669,Young Detective Dee: Rise of the Sea Dragon (D...,Action|Adventure|Drama|Fantasy|Mystery|IMAX,NaN
11061,45758,Lady on a Train (1945),Comedy|Crime|Film-Noir|Mystery|Romance|Thriller,NaN
7089,7201,Crime Spree (2003),Comedy|Crime,caper chase comedy corruption creativity crime...
24445,115821,Shakespeare's Globe: Henry V (2013),Drama,NaN
6568,6678,"Handmaid's Tale, The (1990)",Drama|Sci-Fi,adaptation adapted from:book based on a book b...
12615,59178,Tarzan Escapes (1936),Action|Adventure,NaN
23078,110061,Two Queens and One Consort (Twee vorstinnen en...,Drama,NaN
19001,94496,Columbus Circle (2012),Crime|Mystery|Thriller,NaN
7448,7749,Weekend (a.k.a. Le Week-end) (Week End) (1967),Drama,absurd adultery affectionate allegory art art ...


In [ ]:
def add_genres_to_tag(row):
    if pd.isnull(row['tag']):
        return row['genres'].replace("|", " ")     # If we took the "how = 'left' or 'right' ", this condition was supposed to meet
    else:
        return row['tag'] + " " + row['genres'].replace("|", " ")

In [ ]:
# final_df['tag'] = final_df.apply(lambda row: add_genres_to_tag(row), axis=1)
final_df['tag'] = final_df['tag'].fillna('')

In [ ]:
final_df['tag'][0]


'3d action adventure affectionate animal movie animals animated animation based on book cartoon cgi chase childhood children classic clever colourful comedy computer animation cool creativity cute cute! destiny dialogue disney disney animated feature entertaining excellent excellent script fairy tale family fantasy feel good movie feel-good friendship fun fun movie funny good good soundtrack great great ending great movie heartwarming highly quotable humor humorous imagination imdb top 250 interesting kids kids and family light mentor monsters nostalgic original original plot oscar (best animated feature) oscar (best directing) oscar winner pixar pixar animation quotable runaway sentimental simple story storytelling sweet talking animals talky technology touching toys unlikely friendships very funny very good visual whimsical witty'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [ ]:
cvec = CountVectorizer(stop_words = 'english')
cvec.fit_transform(final_df['tag'])

<27280x1124 sparse matrix of type '<class 'numpy.int64'>'
	with 527019 stored elements in Compressed Sparse Row format>

In [ ]:
bow = cvec.fit_transform(final_df['tag'])

In [ ]:
titles = movies_df['title'].tolist()[:final_df.shape[0]]

In [ ]:
bow_df = pd.DataFrame(bow.toarray(), index= titles).fillna(0)

In [ ]:
bow_df.shape

(27280, 1124)

In [ ]:
tfvec = TfidfVectorizer(stop_words='english')
tfidf = tfvec.fit_transform(final_df['tag'])

In [ ]:
tfidf_df = pd.DataFrame(tfidf.toarray(), index= titles).fillna(0)

In [ ]:
tfidf_df.tail(5)

,0,1,2,3,4,5,6,7,8,9,...,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123
The Pirates (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rentun Ruusu (2001),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Innocence (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Babylon (2021),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Decision to Leave (2022),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarityMatrix = cosine_similarity(bow_df)

In [ ]:
bow_similarity_df = pd.DataFrame(similarityMatrix, index=bow_df.index, columns=bow_df.index)


In [ ]:
bow_similarity_df.head()


,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Brother Bear 2 (2006),No More School (2000),Forklift Driver Klaus: The First Day on the Job (2001),Kein Bund für's Leben (2007),"Feuer, Eis & Dosenbier (2002)",The Pirates (2014),Rentun Ruusu (2001),Innocence (2014),Babylon (2021),Decision to Leave (2022)
Toy Story (1995),0.072539,0.006218,0.0,0.049379,0.016623,0.009235,0.019964,0.023994,0.000000,0.008543,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jumanji (1995),0.006218,0.097015,0.0,0.014815,0.000000,0.000000,0.011980,0.014398,0.081954,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grumpier Old Men (1995),0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Waiting to Exhale (1995),0.049379,0.014815,0.0,0.176471,0.039606,0.000000,0.047565,0.057166,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Father of the Bride Part II (1995),0.016623,0.000000,0.0,0.039606,0.040000,0.000000,0.016013,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tf_similarityMatrix = cosine_similarity(tfidf_df)

In [ ]:
tfidf_similarity_df = pd.DataFrame(tf_similarityMatrix, index=tfidf_df.index, columns=tfidf_df.index)

In [ ]:
tfidf_similarity_df.tail(4)

In [ ]:
def recommend_bow(movie):
    movie_index = bow_similarity_df.index.get_loc(movie)
    top_10 = bow_similarity_df.iloc[movie_index].sort_values(ascending=False)[1:11]
    print(f'Top 10 similar movies to {movie}:')
    print(top_10)

In [ ]:
recommend_bow('Punch-Drunk Love (2002)')


In [ ]:
def recommend_tfidf(movie):
    if movie in tfidf_similarity_df.index:
        movie_index = tfidf_similarity_df.index.get_loc(movie)
        top_10 = tfidf_similarity_df.iloc[movie_index].sort_values(ascending=False)[1:11]
        print(f'Top 10 similar movies to {movie}:')
        print(top_10)
    else:
        print(f"Sorry, the movie '{movie}' is not in the dataset.")

In [ ]:
recommend_tfidf('Dark Knight Rises, The (2012)')


In [ ]:
print(bow_similarity_df.columns)
print(bow_similarity_df.index)

In [ ]:
movie_index = bow_similarity_df.index.get_loc("Dark Knight Rises, The (2012)")
print("BOW Similarity:")
print(bow_similarity_df.iloc[movie_index].sort_values(ascending=False)[:20])

movie_index = tfidf_similarity_df.index.get_loc("Dark Knight Rises, The (2012)")
print("\nTF-IDF Similarity:")
print(tfidf_similarity_df.iloc[movie_index].sort_values(ascending=False)[:20])

In [ ]:
# 데이터 크기 확인
print("Original movies count:", len(movies_df))
print("Final dataset count:", len(final_df))
print("Grouped tags count:", len(grouped_tags))


In [ ]:
!pip install annoy==1.17.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 kB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.1-cp310-cp310-linux_x86_64.whl size=551245 sha256=233d169ea4249ae317fdd0ca670b04421d0b0e7c0fe3888db7a8452383303aa5
  Stored in directory: /root/.cache/pip/wheels/8a/e1/f6/cd65e222d475c5ade306a766fb34c8da2f3f96c01de2a6602a
Successfully built annoy


1. TF-IDF 버전

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from annoy import AnnoyIndex

# 데이터 로드 및 전처리
movies_df = pd.read_csv('/content/movie.csv')
genome_tags_df = pd.read_csv('/content/genome_tags.csv')
genome_scores_df = pd.read_csv('/content/genome_scores.csv')

# 태그 데이터 병합
merged_genome = genome_scores_df.merge(genome_tags_df, on='tagId', how='left')
top_tags = merged_genome[merged_genome['relevance'] > 0.5]
grouped_tags = top_tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

# 최종 데이터프레임 생성 (left join 사용)
final_df = movies_df.merge(grouped_tags, on='movieId', how='left')
final_df = final_df[['movieId', 'title', 'genres', 'tag']]

# NaN 값 처리
final_df['tag'] = final_df['tag'].fillna('')

# TF-IDF 벡터화
tfvec = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfvec.fit_transform(final_df['tag'])

# 영화 제목 리스트 생성
titles = final_df['title'].tolist()

# Annoy 인덱스 생성
def create_annoy_index(tfidf_matrix, n_trees=100):
    vector_dimension = tfidf_matrix.shape[1]
    annoy_index = AnnoyIndex(vector_dimension, 'angular')

    # 벡터 추가
    for i, vec in enumerate(tfidf_matrix.toarray()):
        annoy_index.add_item(i, vec)

    # 인덱스 빌드
    annoy_index.build(n_trees)
    return annoy_index

# Annoy 인덱스 생성
annoy_index = create_annoy_index(tfidf_matrix)

# 영화 제목으로 인덱스 찾기 위한 매핑 생성
title_to_index = {title: idx for idx, title in enumerate(titles)}
index_to_title = {idx: title for idx, title in enumerate(titles)}

# 추천 함수
def recommend_movies(movie_title, n_recommendations=10):
    try:
        # 영화 인덱스 찾기
        movie_idx = title_to_index[movie_title]

        # 유사한 영화 찾기
        similar_indices = annoy_index.get_nns_by_item(movie_idx, n_recommendations + 1)

        # 자기 자신 제외
        similar_indices = [idx for idx in similar_indices if idx != movie_idx]

        print(f'\nTop {n_recommendations} movies similar to "{movie_title}":')
        for idx in similar_indices[:n_recommendations]:
            # 유사도 점수 계산 (선택적)
            similarity = 1 - annoy_index.get_distance(movie_idx, idx)
            print(f"{index_to_title[idx]} (Similarity: {similarity:.3f})")

    except KeyError:
        print(f"Movie '{movie_title}' not found in the database.")


In [ ]:
recommend_movies("Dark Knight Rises, The (2012)", 20)



Top 20 movies similar to "Dark Knight Rises, The (2012)":
Batman Begins (2005) (Similarity: 0.428)
V for Vendetta (2006) (Similarity: 0.254)
Batman: The Dark Knight Returns, Part 1 (2012) (Similarity: 0.238)
Batman: The Dark Knight Returns, Part 2 (2013) (Similarity: 0.217)
Batman/Superman Movie, The (1998) (Similarity: 0.211)
Batman: Under the Red Hood (2010) (Similarity: 0.190)
Unbreakable (2000) (Similarity: 0.166)
Batman Beyond: Return of the Joker (2000) (Similarity: 0.158)
Batman: Year One (2011) (Similarity: 0.155)
Sin City (2005) (Similarity: 0.127)
300 (2007) (Similarity: 0.084)
Kick-Ass (2010) (Similarity: 0.077)
Crow, The (1994) (Similarity: 0.071)
3:10 to Yuma (2007) (Similarity: 0.055)
Collateral (2004) (Similarity: 0.031)
Edge of Tomorrow (2014) (Similarity: 0.018)
Professional, The (Le professionnel) (1981) (Similarity: 0.012)
Town, The (2010) (Similarity: -0.004)
Girl with the Dragon Tattoo, The (2011) (Similarity: -0.006)
Storm of the Century (1999) (Similarity: -0.00

In [ ]:
# 영화 추천 예시
recommend_movies("Dark Knight, The (2008)")
recommend_movies("Inception (2010)")
recommend_movies("Mother (Madeo) (2009)")
recommend_movies("There Will Be Blood (2007)")
recommend_movies("Thirst (Bakjwi) (2009)")




Top 10 movies similar to "Dark Knight, The (2008)":
Batman Begins (2005) (Similarity: 0.500)
Dark Knight Rises, The (2012) (Similarity: 0.424)
Batman/Superman Movie, The (1998) (Similarity: 0.253)
Gladiator (2000) (Similarity: 0.197)
V for Vendetta (2006) (Similarity: 0.181)
Sin City (2005) (Similarity: 0.180)
Unforgiven (1992) (Similarity: 0.158)
Untouchables, The (1987) (Similarity: 0.157)
Departed, The (2006) (Similarity: 0.155)
Road to Perdition (2002) (Similarity: 0.155)

Top 10 movies similar to "Inception (2010)":
Memento (2000) (Similarity: 0.228)
Source Code (2011) (Similarity: 0.224)
Open Your Eyes (Abre los ojos) (1997) (Similarity: 0.213)
Interstellar (2014) (Similarity: 0.199)
Prestige, The (2006) (Similarity: 0.197)
The Butterfly Effect (2004) (Similarity: 0.171)
Minority Report (2002) (Similarity: 0.169)
Twelve Monkeys (a.k.a. 12 Monkeys) (1995) (Similarity: 0.167)
Run Lola Run (Lola rennt) (1998) (Similarity: 0.166)
Predestination (2014) (Similarity: 0.164)

Top 10 mov

In [ ]:
def recommend_multiple_movies(movie_list, n_recommendations=4):
    # 결과를 저장할 딕셔너리
    recommendations = {}

    for movie in movie_list:
        try:
            # 영화 인덱스 찾기
            movie_idx = title_to_index[movie]

            # 유사한 영화 찾기
            similar_indices = annoy_index.get_nns_by_item(movie_idx, n_recommendations + 1)

            # 자기 자신 제외
            similar_indices = [idx for idx in similar_indices if idx != movie_idx]

            # 각 추천 영화의 유사도 점수와 함께 저장
            for idx in similar_indices[:n_recommendations]:
                similar_movie = index_to_title[idx]
                if similar_movie not in recommendations:
                    similarity = 1 - annoy_index.get_distance(movie_idx, idx)
                    recommendations[similar_movie] = {
                        'score': similarity,
                        'based_on': movie
                    }

        except KeyError:
            print(f"Movie '{movie}' not found in the database.")

    # 유사도 점수로 정렬
    sorted_recommendations = sorted(
        recommendations.items(),
        key=lambda x: x[1]['score'],
        reverse=True
    )

    print("\nTop Recommendations based on your movie list:")
    print("=" * 80)
    for movie, data in sorted_recommendations:
        print(f"{movie:<50} (Score: {data['score']:.3f}, Based on: {data['based_on']})")

# 사용 예시
favorite_movies = [
    "Dark Knight, The (2008)",
    "Inception (2010)",
    "Mother (Madeo) (2009)",
    "There Will Be Blood (2007)",
    "Thirst (Bakjwi) (2009)"
]

recommend_multiple_movies(favorite_movies)


Top Recommendations based on your movie list:
Batman Begins (2005)                               (Score: 0.500, Based on: Dark Knight, The (2008))
Dark Knight Rises, The (2012)                      (Score: 0.424, Based on: Dark Knight, The (2008))
No Country for Old Men (2007)                      (Score: 0.296, Based on: There Will Be Blood (2007))
In the Bedroom (2001)                              (Score: 0.256, Based on: There Will Be Blood (2007))
Batman/Superman Movie, The (1998)                  (Score: 0.253, Based on: Dark Knight, The (2008))
Unforgiven (1992)                                  (Score: 0.236, Based on: There Will Be Blood (2007))
Assassination of Jesse James by the Coward Robert Ford, The (2007) (Score: 0.230, Based on: There Will Be Blood (2007))
Memento (2000)                                     (Score: 0.228, Based on: Inception (2010))
Source Code (2011)                                 (Score: 0.224, Based on: Inception (2010))
Open Your Eyes (Abre los ojos)

2. BERT 버전

In [ ]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from annoy import AnnoyIndex

# BERT 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# 데이터 로드 및 전처리 (기존과 동일)
movies_df = pd.read_csv('/content/movie.csv')
genome_tags_df = pd.read_csv('/content/genome_tags.csv')
genome_scores_df = pd.read_csv('/content/genome_scores.csv')

merged_genome = genome_scores_df.merge(genome_tags_df, on='tagId', how='left')
top_tags = merged_genome[merged_genome['relevance'] > 0.5]
grouped_tags = top_tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

final_df = movies_df.merge(grouped_tags, on='movieId', how='left')
final_df = final_df[['movieId', 'title', 'genres', 'tag']]
final_df['tag'] = final_df['tag'].fillna('')

# BERT 임베딩 생성 함수
def get_bert_embedding(text, max_length=512):
    inputs = tokenizer(text, return_tensors='pt', max_length=max_length,
                      truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# 전체 데이터에 대한 BERT 임베딩 생성
embeddings = []
for text in final_df['tag']:
    embedding = get_bert_embedding(text)
    embeddings.append(embedding)

# 임베딩을 numpy 배열로 변환
embeddings_matrix = np.vstack(embeddings)

# 영화 제목 리스트 생성
titles = final_df['title'].tolist()

# Annoy 인덱스 생성
def create_annoy_index(embeddings_matrix, n_trees=100):
    vector_dimension = embeddings_matrix.shape[1]
    annoy_index = AnnoyIndex(vector_dimension, 'angular')

    for i, vec in enumerate(embeddings_matrix):
        annoy_index.add_item(i, vec)

    annoy_index.build(n_trees)
    return annoy_index

# Annoy 인덱스 생성
annoy_index = create_annoy_index(embeddings_matrix)

# 영화 제목으로 인덱스 찾기 위한 매핑 생성
title_to_index = {title: idx for idx, title in enumerate(titles)}
index_to_title = {idx: title for idx, title in enumerate(titles)}

# 추천 함수
def recommend_movies(movie_title, n_recommendations=10):
    try:
        movie_idx = title_to_index[movie_title]
        similar_indices = annoy_index.get_nns_by_item(movie_idx, n_recommendations + 1)
        similar_indices = [idx for idx in similar_indices if idx != movie_idx]

        print(f'\nTop {n_recommendations} movies similar to "{movie_title}":')
        for idx in similar_indices[:n_recommendations]:
            similarity = 1 - annoy_index.get_distance(movie_idx, idx)
            print(f"{index_to_title[idx]} (Similarity: {similarity:.3f})")

    except KeyError:
        print(f"Movie '{movie_title}' not found in the database.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
def recommend_multiple_movies(movie_list, n_recommendations=4):
    # 결과를 저장할 딕셔너리
    recommendations = {}

    for movie in movie_list:
        try:
            # 영화 인덱스 찾기
            movie_idx = title_to_index[movie]

            # 유사한 영화 찾기
            similar_indices = annoy_index.get_nns_by_item(movie_idx, n_recommendations + 1)

            # 자기 자신과 이미 입력된 영화들 제외
            similar_indices = [idx for idx in similar_indices
                             if idx != movie_idx and index_to_title[idx] not in movie_list]

            # 각 추천 영화의 유사도 점수와 함께 저장
            for idx in similar_indices[:n_recommendations]:
                similar_movie = index_to_title[idx]
                if similar_movie not in recommendations:
                    similarity = 1 - annoy_index.get_distance(movie_idx, idx)
                    recommendations[similar_movie] = {
                        'score': similarity,
                        'based_on': movie
                    }
                # 이미 추천 목록에 있는 경우, 더 높은 유사도 점수로 업데이트
                else:
                    new_similarity = 1 - annoy_index.get_distance(movie_idx, idx)
                    if new_similarity > recommendations[similar_movie]['score']:
                        recommendations[similar_movie] = {
                            'score': new_similarity,
                            'based_on': movie
                        }

        except KeyError:
            print(f"Movie '{movie}' not found in the database.")

    # 유사도 점수로 정렬
    sorted_recommendations = sorted(
        recommendations.items(),
        key=lambda x: x[1]['score'],
        reverse=True
    )

    print("\nTop Recommendations based on your movie list:")
    print("=" * 80)
    for movie, data in sorted_recommendations:
        print(f"{movie:<50} (Score: {data['score']:.3f}, Based on: {data['based_on']})")

# 사용 예시
favorite_movies = [
    "Dark Knight, The (2008)",
    "Inception (2010)",
    "Mother (Madeo) (2009)",
    "There Will Be Blood (2007)",
    "Thirst (Bakjwi) (2009)"
]

recommend_multiple_movies(favorite_movies)


Top Recommendations based on your movie list:
Dark Knight Rises, The (2012)                      (Score: 0.852, Based on: Dark Knight, The (2008))
Memories of Murder (Salinui chueok) (2003)         (Score: 0.821, Based on: Mother (Madeo) (2009))
Confessions (Kokuhaku) (2010)                      (Score: 0.819, Based on: Mother (Madeo) (2009))
Batman Begins (2005)                               (Score: 0.819, Based on: Dark Knight, The (2008))
Unforgiven (1992)                                  (Score: 0.807, Based on: There Will Be Blood (2007))
Nightcrawler (2014)                                (Score: 0.803, Based on: Mother (Madeo) (2009))
Hunt, The (Jagten) (2012)                          (Score: 0.803, Based on: Mother (Madeo) (2009))
21 Grams (2003)                                    (Score: 0.800, Based on: There Will Be Blood (2007))
Prestige, The (2006)                               (Score: 0.798, Based on: Inception (2010))
Memento (2000)                                     (S

In [ ]:
recommend_movies("Dark Knight Rises, The (2012)", 20)



Top 20 movies similar to "Dark Knight Rises, The (2012)":
Batman Begins (2005) (Similarity: 0.853)
Dark Knight, The (2008) (Similarity: 0.852)
Iron Man (2008) (Similarity: 0.777)
V for Vendetta (2006) (Similarity: 0.764)
Die Hard (1988) (Similarity: 0.753)
Unbreakable (2000) (Similarity: 0.751)
Lord of the Rings: The Return of the King, The (2003) (Similarity: 0.750)
Guardians of the Galaxy (2014) (Similarity: 0.749)
Lord of the Rings: The Two Towers, The (2002) (Similarity: 0.748)
Bittersweet Life, A (Dalkomhan insaeng) (2005) (Similarity: 0.748)
Indiana Jones and the Last Crusade (1989) (Similarity: 0.737)
Terminator, The (1984) (Similarity: 0.737)
Usual Suspects, The (1995) (Similarity: 0.737)
300 (2007) (Similarity: 0.737)
Sin City (2005) (Similarity: 0.737)
Watchmen (2009) (Similarity: 0.737)
Collateral (2004) (Similarity: 0.736)
Lord of the Rings: The Fellowship of the Ring, The (2001) (Similarity: 0.735)
Aliens (1986) (Similarity: 0.735)
Reservoir Dogs (1992) (Similarity: 0.735

In [ ]:
import os
import pickle
from annoy import AnnoyIndex

def save_content_model(annoy_index, final_df, model_dir='/content/content_model'):
    """
    콘텐츠 기반 추천 모델과 관련 데이터를 저장하는 함수
    """
    os.makedirs(model_dir, exist_ok=True)

    # Annoy 인덱스 저장
    annoy_index.save(os.path.join(model_dir, 'movie_similarity.ann'))

    # 필요한 데이터 저장
    model_data = {
        'titles': final_df['title'].tolist(),
        'title_to_index': title_to_index,
        'index_to_title': index_to_title,
        'movie_data': final_df[['movieId', 'title', 'genres']].to_dict('records')  # 영화 정보도 저장
    }

    with open(os.path.join(model_dir, 'content_model_data.pkl'), 'wb') as f:
        pickle.dump(model_data, f)

    print(f"Content-based model saved in {model_dir}")

def get_content_recommendations(movie_list, model_data, annoy_index, n_recommendations=10):
    """
    콘텐츠 기반 추천을 수행하고 표준화된 형식으로 결과를 반환하는 함수
    """
    recommendations = {}

    for movie in movie_list:
        try:
            movie_idx = model_data['title_to_index'][movie]
            similar_indices = annoy_index.get_nns_by_item(movie_idx, n_recommendations + 1)

            for idx in similar_indices:
                if idx != movie_idx:
                    movie_title = model_data['index_to_title'][idx]
                    if movie_title not in recommendations and movie_title not in movie_list:
                        similarity = 1 - annoy_index.get_distance(movie_idx, idx)
                        movie_info = next(m for m in model_data['movie_data']
                                        if m['title'] == movie_title)

                        recommendations[movie_title] = {
                            'movieId': movie_info['movieId'],
                            'title': movie_title,
                            'genres': movie_info['genres'],
                            'content_score': similarity,
                            'based_on': movie
                        }

        except KeyError:
            print(f"Movie '{movie}' not found in the database.")

    # 결과를 점수 기준으로 정렬
    sorted_recommendations = sorted(
        recommendations.values(),
        key=lambda x: x['content_score'],
        reverse=True
    )

    return sorted_recommendations[:n_recommendations]

save_content_model(annoy_index, final_df)


Content-based model saved in /content/content_model
